In [ ]:
from warnings import warn
from statistics import mean
from tqdm import tqdm
import requests as reqs
import numpy as np
import pandas as pd

Данные Tutu.ru - список железнодорожных станций отправления и прибытия:  
https://support.travelpayouts.com/hc/ru/articles/115001440551-Данные-от-Tutu-ru

In [ ]:
tutu_routes = pd.read_excel('tutu_routes.xlsx') #
pairs = list(zip(tutu_routes.departure_station_id, tutu_routes.arrival_station_id))

In [ ]:
def get_api_request(request: str, params: dict = {},):
    """Returns formatted request response
    """
    REQUEST_BASE = "https://suggest.travelpayouts.com/"

    request_form = f"{REQUEST_BASE}/{request}?&service=tutu_trains"

    response = reqs.get(request_form, params=params)
    if response.ok:
        return response.json()
    else:
        response.raise_for_status()

def get_trains_info(st1: str, st2: str):

    request = 'search'
    params = {"term": st1, "term2": st2}
    return get_api_request(request, params)

In [ ]:

errors = []
time_list = []
for pair in tqdm(pairs):
    try:
        scraped = get_trains_info(pair[0], pair[1])
        trips_time = []
        try:
            for trip in scraped['trips']:
                trips_time.append(int(trip['travelTimeInSeconds']))
            time_list.append(mean(trips_time))
        except: time_list.append(np.nan)
    except: time_list.append(np.nan), errors.append(pair)

In [ ]:
tutu_routes['time'] = time_list

In [ ]:
tutu_routes.to_excel('routes.xlsx')